In [9]:
import numpy as np


def threshold_crossing(u, t, threshold, inactive, cross_num, act_t):
    threshold_cross = (u >= threshold) & inactive
            
    inactive[threshold_cross] = False
    cross_num[threshold_cross] += 1

    coords = np.argwhere(threshold_cross[np.newaxis, :, :])
    coords[:, 0] = cross_num[threshold_cross]

    if cross_num.max() >= act_t.shape[0]:
        act_t = np.vstack((act_t, 
                                -np.ones((1, *act_t.shape[1:]))))

    act_t[tuple(coords.T)] = t

    back_cross = ~inactive & (u < threshold)
    inactive[back_cross] = True

    return act_t


u = np.random.rand(100, 100)
t = 10

threshold = 0.5
inactive = np.full_like(u, True, dtype=bool)
cross_num = np.zeros_like(u)
act_t = -np.ones((1, *u.shape))

%timeit threshold_crossing(u, t, threshold, inactive, cross_num, act_t)

67.2 µs ± 503 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
coords = np.argwhere(u >= threshold)

%timeit act_t[tuple(coords.T)] = t

867 µs ± 20.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
%timeit np.where(u >= threshold, t, act_t)

23.9 µs ± 323 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
%timeit act_t[u >= threshold] = t

32.2 µs ± 504 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [19]:
act_t = -np.ones(u.shape)
%timeit res = np.where(u >= threshold, t, act_t)

24.6 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [30]:
cross_mask = u >= threshold

In [35]:
%timeit np.where((act_t > -1) & cross_mask)

64.2 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
act_t = -np.ones((100, *u.shape))

%timeit np.vstack((act_t, -np.ones((1, *act_t.shape[1:]))))

697 µs ± 7.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
def track(u, t, threshold, activated, act_t, amount):
    # Calculate updated activation times where the threshold is crossed
    updated_array = np.where((act_t[-1] < 0) & (u > threshold), t, -1)

    # Update the amount of activations for cells that cross the threshold again
    if np.any((activated == False) & (act_t[-1] > 0) & (u > threshold)):
        amount = np.where(
            (activated == False) & (act_t[-1] > 0) & (u > threshold),
            amount + 1, amount
        )
        # If any cell has been activated more times than the current activation list length, append a new array
        if np.any(amount > len(act_t)):
            act_t.append(updated_array)
    else:
        # Update the last recorded activation times with new activations
        act_t[-1] = np.where(updated_array > 0, updated_array, act_t[-1])

    # Update the activation status of cells
    activated[1:-1, 1:-1] = np.where(
        (u[1:-1, 1:-1] > threshold) & (activated[1:-1, 1:-1] == False),
        True, activated[1:-1, 1:-1]
    )
    # Reset the activation status if the potential drops below the threshold
    activated[1:-1, 1:-1] = np.where(
        (u[1:-1, 1:-1] <= threshold) & (activated[1:-1, 1:-1] == True),
        False, activated[1:-1, 1:-1]
    )


%timeit track(u, t, threshold, inactive, act_t, cross_num)

121 µs ± 3.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
